In [1]:
#CLOCKSPEED = 0.75

import airsim
import numpy as np
import cv2
import os
import datetime
from PIL import Image

import nbimporter
import Classifier
import matplotlib.pyplot as plt
import timeit
import time

depth_det = 10
depth_cf = 40
stacksize = 5

detectable = ['No_gesture', 'Swiping_Left', 'Swiping_Right', 'Stop_Sign', 'Pulling_Hand_In'] 

Importing Jupyter notebook from Classifier.ipynb


In [2]:
def GetCurrentFrame():
    responses = client.simGetImages([
        airsim.ImageRequest("1", airsim.ImageType.Scene, False, False)])  #scene vision image in uncompressed RGB array

    response = responses[0]
    img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) # get numpy array
    img_rgb = img1d.reshape(response.height, response.width, 3) # reshape array to 3 channel image array H X W X 3
    #cv2.imwrite('fr.png',img_rgb)
    
    return img_rgb #as nparray

In [3]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath, map_location='cpu')

    my_dict = checkpoint['state_dict']
    my_dict_copy = my_dict.copy()

    for key in my_dict_copy.keys():
        newkey = key[7:]
        my_dict[newkey] = my_dict.pop(key)

    checkpoint['state_dict'] = my_dict 
    
    model = resnet.resnet10(
        num_classes=2,
        shortcut_type='B',
        cardinality=32,
        sample_size=112,
        sample_duration=depth_det)

    model.load_state_dict(checkpoint['state_dict'])

    model.eval()
    return model

In [4]:
def getTransforms():
    scales = [1]
    for i in range(1, 5):
        scales.append(scales[-1] * 0.84089641525)
        
    norm_method = Normalize(get_mean(norm_value), [1, 1, 1])
            
    spatial_transform = spatial_transforms.Compose([
        Scale(112),
        CenterCrop(112),
        ToTensor(norm_value), norm_method 
    ])

    return spatial_transform

In [5]:
#TURN THE 37 FRAMES FROM ./TESTVID INTO TENSOR INPUT <1,3,32,112,112>
def TransformData(spatial_transform, frameST):
    a = []
    for frame in frameST:
        image = torchvision.transforms.ToPILImage()(frame)
        a.append(image)

    spatial_transform.randomize_parameters()
    input_spat = [spatial_transform(image) for image in a]
    
    im_dim = input_spat[0].size()[-2:]
    clip = torch.cat(input_spat, 0).view((duration, -1) + im_dim).permute(1, 0, 2, 3)
    input = clip[None, :, :, :, :] #Add 5th Dimension; batchSize=1
    print(input.shape)
    return input

In [6]:
def CropHuman(frame, show=False):
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    frame = cv2.resize(frame, (768, 288))
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    boxes, weights = hog.detectMultiScale(frame, winStride=(8,8) )
    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])
    for (xA, yA, xB, yB) in boxes:
        cv2.rectangle(frame, (xA, yA), (xB, yB),
                        (0, 255, 0), 2)

    if show:
        plt.imshow(frame, cmap = 'gray', interpolation = 'bicubic')
        plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
        plt.show()
    
    #return(boxes[0][0],boxes[0][1],boxes[0][2],boxes[0][3])
    return(boxes)

In [7]:
def human_detector(fr, verbose=False):
    cv2.imwrite('hum_det.png',fr)
    boxes = CropHuman(fr, verbose)
    if boxes.size == 0:
        return 0,0,0,0
    else:
        x = boxes[0][0] * 2
        y = boxes[0][1] * 2
        xF = boxes[0][2] * 2
        yF = boxes[0][3] * 2
        return x,y,xF,yF

In [8]:
def CropLastX(vstreamshort,x,y,xF,yF):
    newstream = []
    for fr in vstreamshort:
        cv2.imwrite('OL1.png',fr) ######
        fr = fr[y:yF,x:xF]
        cv2.imwrite('OL2.png',fr) ##########
        height = fr.shape[0]
        width = fr.shape[1]
        #fr = fr[int(round(height/14)):int(round(height/2.5)),int(round(width/14)):int(round(width*9/10))]
        fr = fr[int(round(height/7)):int(round(height/3)),int(round(width/9)):int(round(width*4/5))]
        cv2.imwrite('OL3.png',fr) ########
        newstream.append(fr)
    return newstream

In [9]:
def Detector(frameST):
    #model = load_checkpoint('NameOfPTH')
    #input_det = TransformData(getTransforms(), frameST)
    #model.eval()
    #out_det = model(input_det)
    
    #classes = [0,1] #No Gest, Gest
    #_, index = torch.max(out_det, 1)
    #percentage = torch.nn.functional.softmax(out_det, dim=1)[0]
    #print(classes[index[0]], percentage[index[0]].item())
    
    #return(classes[index[0]])
    return True
    

In [10]:
def actuator(gesture):
    if gesture == 'No_gesture': #break
        #car_straightsmall()
        car_break()
    elif gesture == 'Swiping_Left': #go right
        car_straightsmall()
        car_right()
    elif gesture == 'Swiping_Right': #go left
        car_straightsmall()
        car_left()
    elif gesture == 'Stop_Sign': #break
        car_break()
    elif gesture == 'Pulling_Hand_In': #go straight
        import random
        num = random.randint(0,3)
        if num == 3:
            car_straightsmall()
            car_right()
        else:
            car_straight()

In [11]:
def car_break():
    car_controls.brake = 1
    client.setCarControls(car_controls)
    print("Apply brakes")
    time.sleep(1)   # let car drive a bit
    car_controls.brake
    client.reset()
    client.enableApiControl(False)
    print("RESET BISH")
    trigger = True
    print("DONE RESETTING")

In [12]:
def car_right():
    car_controls.throttle = 1
    car_controls.steering = 0.98
    client.setCarControls(car_controls)
    print("Go Forward, steer right")
    time.sleep(3.12)   # let car drive a bit
    car_straight()

In [13]:
def car_left():
    car_controls.throttle = 1.0
    car_controls.steering = -0.7
    client.setCarControls(car_controls)
    print("Go Forward, steer left")
    time.sleep(3.82)   # let car drive a bit
    car_straight()

In [14]:
def car_straight():
    car_controls.throttle = 1.0
    car_controls.steering = 0
    client.setCarControls(car_controls)
    print("Go Forward")
    time.sleep(6)   # let car drive a bit
    car_break()

In [17]:
def car_straightsmall():
    car_controls.throttle = 1.0
    car_controls.steering = 0
    client.setCarControls(car_controls)
    print("Go Forward")
    time.sleep(3.2)   # let car drive a bit

In [18]:
def DetectAndDrive():
    global client, car_controls
    client = airsim.CarClient()
    client.confirmConnection()
    client.enableApiControl(True)
    car_controls = airsim.CarControls()
    
    videostream = []
    
    while len(videostream) < depth_cf:
        try:
            videostream.append(GetCurrentFrame())
        except ValueError:
            continue #skip to next frame
        if len(videostream) >= depth_cf:
            if len(videostream) % stacksize == 0: #every X frames run human_detector
                print("Got enough frames. Analyzing")
                x,y,xF,yF = human_detector(videostream[len(videostream)-1], False)
                if x or y or xF or yF: #Only continue if human detected
                    croppedstream_recent = CropLastX(videostream[-depth_cf:],x,y,xF,yF) #Crop last X frames of VS
                    exists = Detector(croppedstream_recent[-depth_det:]) #See if gesture detected
                    if exists:
                        print('Classification Time')
                        classes, indices, percentages = Classifier.Classify(croppedstream_recent, True)
                        for i in range(0,26):
                            gesture = classes[indices[0][i]]
                            if gesture not in detectable:
                                continue
                            else:
                                print(gesture)
                                actuator(gesture)
                                break

In [ ]:
if __name__ == '__main__':  
    global trigger
    trigger = True
    
    while 1:
        DetectAndDrive()
        #print(trigger)
        #if trigger:s
        #    print("GOING")
        #    DetectAndDrive()
        #    trigger = False

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



c:\users\jcruz\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


Got enough frames. Analyzing
Classification Time


C:\Users\jcruz\Downloads\GestureRecognition\airsim\resnext.py:121: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


Pulling_Hand_In
Go Forward
Apply brakes
RESET BISH
DONE RESETTING
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



c:\users\jcruz\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


Got enough frames. Analyzing
Classification Time
Pulling_Hand_In
Go Forward
Apply brakes
RESET BISH
DONE RESETTING
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

Got enough frames. Analyzing
Classification Time
Pulling_Hand_In
Go Forward
Apply brakes
RESET BISH
DONE RESETTING
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

Got enough frames. Analyzing
Classification Time
Swiping_Right
Go Forward
Go Forward, steer left
Go Forward
Apply brakes
RESET BISH
DONE RESETTING
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

Got enough frames. Analyzing
Classification Time
Pulling_Hand_In
Go Forward
Go Forward, steer right
Go Forward
Apply brakes
RESET BISH
DONE RESETTING
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

Got enough frames. Analyzing
Classification Time
Pulling_Hand_In
Go Forward
Apply brakes
RESET BISH
DONE RESETTING
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

Got enough frames. Analyzing
Clas